In [1]:
import os
import sys
import itertools
import numpy as np
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from dnc.dnc import DNC
import dnc.utility as uf
from tqdm import tqdm
from feedforward_controller import FeedforwardController
from basic_recurrent_controller import BasicRecurrentController
from focus_recurrent_controller import FocusRecurrentController
from spotlight_recurrent_controller import SpotlightRecurrentController
import time
from feedforward_controller import FeedforwardController
from basic_recurrent_controller import BasicRecurrentController
from focus_recurrent_controller import FocusRecurrentController
from spotlight_recurrent_controller import SpotlightRecurrentController
from circle_recurrent_controller import CircularSpotlightRecurrentController


cifs_path = "/media/data_cifs/DNC_Visual_Reasoning_Results_Logs"


#Remove logging  from previous training runs
os.system("rm {}/*.npy".format(cifs_path))

#Parameters of the task and the training
params = {}
params["timestamp"] = str(int(time.time())) #the  identifier for this test run
params["task"] = "2_square_detect" #specify the task
params["num_iter"] = 20000 #the number of batches to run
params["bsize"] = 10 #the batch size
params["input_side"] = 24 #the length of each side of each image
params["input_size"] = params["input_side"]**2 #the number of pixels
params["num_labels"] = 2 #the number of labels
params["sequence_length"] = 16 #the number of images in the sequence
params["half_max_item"] = 3 #parameter for sd task; Note: if this changes, then so should *sigma_max* in spotlight_recurrent_controller.py
params["memory_words_num"] = 10 #the number of memory words
params["memory_word_size"] = 10#the size of memory words
params["memory_read_heads"] = 1 #the number of read heads
params["print_step"] = 500 #the number of steps between each loss printintg
params["save_step"] = 4000 # the number of steps between each save
params["device"] = "/gpu:0" #Set this to /gpu:0 or /gpu:1 etc if you want to use the gpu instead
params["focus_type"] = "none"
params["loss_type"] = "all_steps"

params["item_position"] = "random" # fixed or random; controls location of items in square_detect, 2_square_detect and sd tasks
params["item_size"] = "random"     # ""; controls size ""




# Import correct controller and define attention attributes
if params["focus_type"] == "none":
    ctrlr = BasicRecurrentController
    get_attributes = lambda c: ([c.W1], [c.W2], [c.W3])
    attr1 = "W1"
    attr2 = "W2"
    attr3 = "W3"
elif params["focus_type"] == "mask" or params["focus_type"] == "rowcol":
    ctrlr = FocusRecurrentController
    get_attributes = lambda c: (c.focus_row, c.focus_col, c.focus_mask)
elif params["focus_type"] == "spotlight":
    ctrlr =  SpotlightRecurrentController
    get_attributes = lambda c: (c.spotlight_row, c.spotlight_col, c.spotlight_sigma)
elif params["focus_type"] == "circular_spotlight":
    ctrlr = CircularSpotlightRecurrentController
    get_attributes = lambda c: (c.spotlight_row, c.spotlight_col, c.spotlight_radius)

# Set loss function

# Loss at all time steps
if params["loss_type"] == "all_steps": 
    params["loss_weightings"] = np.ones(params["sequence_length"])

# Loss at last timestep     
elif params["loss_type"] == "last_step":
    params["loss_weightings"] = np.array([i == (params["sequence_length"] - 1) for i in range(params["sequence_length"])])  

# Loss increasing by timestep
elif params["loss_type"] == "increasing": 
    params["loss_weightings"] = np.arange(params["sequence_length"]) 

assert len(params["loss_weightings"]) == params["sequence_length"], ("Length of loss weights must be equal to sequence length")

#Test
_, _ = uf.make_ims(params)


#Make the directory for this run of the algorithm and save the params to it

os.system("mkdir -p {}/{}".format(cifs_path, params["timestamp"]))
os.system("cp DNC_Visual_Reasoning.py {}/{}/DNC_Visual_Reasoning_snapshot.py".format(cifs_path, params["timestamp"]))
np.save("{}/{}/params.npy".format(cifs_path, params["timestamp"]), params)


#Reset the graph and run the algorithm
tf.reset_default_graph()
with tf.device(params["device"]):
    sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True))
  
    #build the neural computer
    ncomputer = DNC(
        ctrlr,
        input_size=params["input_size"],
        output_size=params["num_labels"],
        sequence_length=params["sequence_length"],
        controller_params={"focus_type":params["focus_type"]},
        memory_words_num=params["memory_words_num"],
        memory_word_size=params["memory_word_size"],
        memory_read_heads=params["memory_read_heads"],
        batch_size=params["bsize"]
    )
    attr1, attr2, attr3 = get_attributes(ncomputer.controller)
    output, loss = ncomputer.get_elementwise_loss(params["loss_weightings"]) 
    
    print "initializing..."
    updt, grads = uf.get_updt(loss)
    init = tf.global_variables_initializer()
    sess.run(init)
    print "initialized!"

    loss_vals = []
    input_vals = []
    output_vals = []
    target_vals = []
    view_vals = []
    attributes = []
    mem = []
       
for i in tqdm(range(params["num_iter"])):

    #Get the data and expected output for this batch
    Input, Target_Output = uf.make_ims(params)

    #Run the  update step

    OUT = sess.run([
    loss,
    output,
    ncomputer.packed_memory_view,
    updt] +  attr1 + attr2 + attr3, 
    feed_dict={
        ncomputer.input_data: Input,
        ncomputer.target_output: Target_Output
    })

    l, o, v = OUT[:3]
    out_attr1 = OUT[4:4 + len(attr1)]
    out_attr2 = OUT[4 + len(attr1):4 + len(attr1) + len(attr2)]
    out_attr3 = OUT[4 + len(attr1) +  len(attr2):4 + len(attr1) + len(attr2) + len(attr3)]


    #Keep track of the values at this timestep
    loss_vals.append(l)
    input_vals.append(Input)
    output_vals += list(o)
    view_vals.append(v)
    target_vals += list(Target_Output)
    mem.append(ncomputer.packed_memory_view)
    attributes.append((np.array(out_attr1), np.array(out_attr2), np.array(out_attr3)))

    #Print the loss and accuracy thus far
    if len(target_vals) % params["print_step"] == 0 and len(target_vals) > 0:
        print "np.array(target_vals).shape", np.array(target_vals).shape
        print "np.array(output_vals).shape", np.array(output_vals).shape

        losses = {}
        losses["loss"] = np.mean(loss_vals[-params["print_step"]:])
        losses["matches"] = np.mean(np.argmax(np.array(output_vals)[-params["print_step"]:, -1], -1) == 
                                 np.argmax(np.array(target_vals)[-params["print_step"]:, -1], -1))

        print "loss", losses["loss"]
        print "matches", losses["matches"]

        np.save("{}/{}/losses_{}.npy".format(cifs_path, params["timestamp"], i), losses)

    #Save the model and the masks generated
    if len(target_vals) % params["save_step"] == 0 and len(target_vals) > 0:
        print "saving for {}".format(i)
        np.save("{}/{}/outputs_{}.npy".format(cifs_path, params["timestamp"], i), output_vals[-50:])
        np.save("{}/{}/targets_{}.npy".format(cifs_path, params["timestamp"], i), target_vals[-50:])
        np.save("{}/{}/inputs_{}.npy".format(cifs_path, params["timestamp"], i), input_vals[-50:])
        np.save("{}/{}/attributes_{}.npy".format(cifs_path, params["timestamp"], i), attributes[-50:])

        #Save the weights of the model - disabled because the model checkpoints are big and bulky 
        # ncomputer.save(sess, 
        #                "{}/{}".format(params["timestamp"]), cifs_path, 
        #                "saved_weights_{}.npy".format(i))



 

/usr/local/lib/python2.7/dist-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
dnc/utility.py:297: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  stamp1[upper_left_corner1[0]:upper_left_corner1[0] + square_side1 , upper_left_corner1[1]:upper_left_corner1[1] + square_side1] = square1
dnc/utility.py:298: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  stamp2[upper_left_corner2[0]:upper_left_corner2[0] + square_side2 , upper_left_corner2[1]:upper_left_corner2[1] + square_side2] = square2
dnc/utility.py:266: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  canvas[upper_left_corner[0]:upper_left_corner[0] + square_side , upper_left_corner[1]:upper_left_corner

True


100%|██████████| 16/16 [00:47<00:00,  3.06s/it]12%|█▎        | 2/16 [00:05<00:40,  2.88s/it]


initializing...
computing gradients...


  0%|          | 0/17 [00:00<?, ?it/s]100%|██████████| 17/17 [00:00<00:00, 198.34it/s]


applying gradients...


  0%|          | 0/20000 [00:00<?, ?it/s]

initialized!


InternalError: Blas SGEMM launch failed : a.shape=(10, 586), b.shape=(586, 256), m=10, n=256, k=586
	 [[Node: controller/shape_inference/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](controller/shape_inference/MatMul/a, controller/layer1_W/read)]]
	 [[Node: sequence_loop/TopKV2_5/k/_361 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_29185_sequence_loop/TopKV2_5/k", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'controller/shape_inference/MatMul', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <mod

In [ ]:

for i in tqdm(range(params["num_iter"])):

    #Get the data and expected output for this batch
    Input, Target_Output = uf.make_ims(params)

    #Run the  update step

    OUT = sess.run([
    loss,
    output,
    ncomputer.packed_memory_view,
    updt] +  attr1 + attr2 + attr3, 
    feed_dict={
        ncomputer.input_data: Input,
        ncomputer.target_output: Target_Output
    })

    l, o, v = OUT[:3]
    out_attr1 = OUT[4:4 + len(attr1)]
    out_attr2 = OUT[4 + len(attr1):4 + len(attr1) + len(attr2)]
    out_attr3 = OUT[4 + len(attr1) +  len(attr2):4 + len(attr1) + len(attr2) + len(attr3)]


    #Keep track of the values at this timestep
    loss_vals.append(l)
    input_vals.append(Input)
    output_vals += list(o)
    view_vals.append(v)
    target_vals += list(Target_Output)
    mem.append(ncomputer.packed_memory_view)
    attributes.append((np.array(out_attr1), np.array(out_attr2), np.array(out_attr3)))

    #Print the loss and accuracy thus far
    if len(target_vals) % params["print_step"] == 0 and len(target_vals) > 0:
        print "np.array(target_vals).shape", np.array(target_vals).shape
        print "np.array(output_vals).shape", np.array(output_vals).shape

        losses = {}
        losses["loss"] = np.mean(loss_vals[-params["print_step"]:])
        losses["matches"] = np.mean(np.argmax(np.array(output_vals)[-params["print_step"]:, -1], -1) == 
                                 np.argmax(np.array(target_vals)[-params["print_step"]:, -1], -1))

        print "loss", losses["loss"]
        print "matches", losses["matches"]

        np.save("{}/{}/losses_{}.npy".format(cifs_path, params["timestamp"], i), losses)

    #Save the model and the masks generated
    if len(target_vals) % params["save_step"] == 0 and len(target_vals) > 0:
        print "saving for {}".format(i)
        np.save("{}/{}/outputs_{}.npy".format(cifs_path, params["timestamp"], i), output_vals[-50:])
        np.save("{}/{}/targets_{}.npy".format(cifs_path, params["timestamp"], i), target_vals[-50:])
        np.save("{}/{}/inputs_{}.npy".format(cifs_path, params["timestamp"], i), input_vals[-50:])
        np.save("{}/{}/attributes_{}.npy".format(cifs_path, params["timestamp"], i), attributes[-50:])

        #Save the weights of the model - disabled because the model checkpoints are big and bulky 
        # ncomputer.save(sess, 
        #                "{}/{}".format(params["timestamp"]), cifs_path, 
        #                "saved_weights_{}.npy".format(i))



 


  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
%matplotlib inline
%load_ext autoreload

%autoreload 2
import os
import sys
import itertools
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from dnc.dnc import DNC
import dnc.utility as uf

# from feedforward_controller import FeedforwardController
from basic_recurrent_controller import BasicRecurrentController
from focus_recurrent_controller import FocusRecurrentController

from tqdm import tqdm
import time
import seaborn as sns
plt.rcParams['figure.figsize'] = (5, 10)

sess = None
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

# TODO: For the DNC-controlled sliding window, just make the "sequence input" to the DNC be a sequence of images. Then
# make the first step of the controller to be to apply the index window on top of the image

In [ ]:
# num_labels = 2
def make_ims(kind, size=8, splits=4):
    if kind == "center":
        Input, _, Target_Output = uf.get_center_bar_images(bsize, size=size, splits=splits, stagger=False)
    elif kind == "right":
        Input, _, Target_Output = uf.get_right_bar_images(bsize, size=size, splits=splits, stagger=False)
    return Input, Target_Output


if not sess is None:
    sess.close()

bsize = 1
input_size = 64
num_labels = 2
sequence_length = 16
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True))

ncomputer = DNC(
    FocusRecurrentController,
    input_size=input_size,
    output_size=num_labels,
    max_sequence_length=sequence_length,
    memory_words_num=10,
    memory_word_size=10,
    memory_read_heads=1,
    batch_size=bsize
)
assert ncomputer.controller.has_recurrent_nn

raw_outputs, memory_views = ncomputer.get_outputs()
output = tf.argmax(raw_outputs[:, sequence_length - 1, :], 1)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(raw_outputs[:, sequence_length - 1, :], 
                                                              ncomputer.target_output_final))

start = time.time()
updt = uf.get_updt(loss)
print time.time() - start

init = tf.global_variables_initializer()
print "initializing..."
sess.run(init)
print "initialized!"


In [ ]:

print_step = 200
losses = []
inputs = []
outputs = []
targets = []
views = []
raw_focuses_row = []
raw_focuses_col = []
focuses = []

for i in tqdm(range(1000000)):
    
    Input, Target_Output = make_ims("right")

    OUT = sess.run([
        loss,
        output,
        memory_views,
        updt] + 
        ncomputer.controller.focus_row +
        ncomputer.controller.focus_col
        , feed_dict={
        ncomputer.input_data: Input,
        ncomputer.target_output_final: Target_Output,
        ncomputer.sequence_length: sequence_length
    })
    l, o, v = OUT[:3]
    fr = OUT[4:4+len(ncomputer.controller.focus_row)]
    fc = OUT[4+len(ncomputer.controller.focus_row):]
    pairs = zip(np.argmax(np.array(fr)[:,0,:], -1), np.argmax(np.array(fr)[:,0,:], -1))

    losses.append(l)
    inputs.append(Input)
    outputs += list(o)
    views.append(v)
    targets += list(np.argmax(Target_Output, axis=-1))
    raw_focuses_row.append(np.array(fc)[:,0,:])
    raw_focuses_col.append(np.array(fc)[:,0,:])
    focuses.append(pairs)
    if len(targets) % print_step == 0 and len(targets) > 0:
        print "loss", np.mean(losses[-print_step:])
        print "matches", np.mean(np.array(targets[-print_step:]) == np.array(outputs[-print_step:]))


In [ ]:
focuses[-1]

In [ ]:
# rup

In [ ]:
# plt.subplot(2,2,1)
# plt.imshow(np.reshape(Input[0][0], (4,4)))
# plt.subplot(2,2,2)
# plt.imshow(np.reshape(Input[0][1], (4,4)))
# plt.subplot(2,2,3)
# plt.imshow(np.reshape(Input[0][2], (4,4)))
# plt.subplot(2,2,4)
# plt.imshow(np.reshape(Input[0][3], (4,4)))

In [ ]:
# focus_row_updater, focus_col_updater, normed_nn_output, rup = sess.run([
#         ncomputer.controller.focus_row_updater,
#         ncomputer.controller.focus_col_updater,
#         ncomputer.controller.nn_output/(1e-4 + tf.reduce_sum(tf.abs(ncomputer.controller.nn_output))), 
#         tf.matmul(ncomputer.controller.nn_output/(1e-4 + tf.reduce_sum(tf.abs(ncomputer.controller.nn_output))), 
#                   ncomputer.controller.focus_row_updater)
        
#     ], feed_dict={
#         ncomputer.input_data: Input,
#         ncomputer.target_output_final: Target_Output,
#         ncomputer.sequence_length: sequence_length
#     })

In [ ]:
# ix = 4
# v = views[ix]
# print targets[ix]
# Input = inputs[ix]

# for i, w, r in zip(range(4), np.squeeze(v['write_weightings'][0]), np.squeeze(v['read_weightings'][0])):
#     plt.subplot(2,2,i + 1)
#     plt.plot(w, label="w")
#     plt.plot(r, label="r")
#     plt.legend()

In [ ]:
# # batch_x, batch_y = mnist.train.next_batch(bsize)
# # Input, Target_Output = get_im_sequence(batch_x, batch_y)
# Input, _, Target_Output = make_ims(bsize, size=im_size, splits=im_splits)

# l, o, v, _ = sess.run([
#     loss,
#     output,
#     memory_views, 
#     updt
# ], feed_dict={
#     ncomputer.input_data: Input,
#     ncomputer.target_output_final: Target_Output,
#     ncomputer.sequence_length: sequence_length
# })
# o

In [ ]:

# with tf.variable_scope("testing4"): 
#     Xf2 = tf.placeholder(tf.float32, [1, 26], name="Xf2")
#     state2 = tf.placeholder(tf.float32, [1, 256], name="state2")
#     out = ncomputer.controller.run_controller_network(Xf2, state2)
#     init = tf.global_variables_initializer()
#     print "initializing..."
#     sess.run(init)
#     print "initialized!"
#     l1_output, l2_output, l3_output, nn_output = sess.run([
#             ncomputer.controller.l1_output,
#             ncomputer.controller.l2_output,
#             ncomputer.controller.l3_output, 
#             ncomputer.controller.nn_output
#         ], feed_dict={
#             Xf2: np.zeros((1, 26)),
#             state2: np.random.random((1, 256))
#         })

In [ ]:
# ncomputer.save(sess, "ckpts", "basic_recurrent_controller_get_lrb_images_task.ckpt")

In [ ]:

#TODO: VISUALIZE GRADIENTS AND MEMORY

















In [ ]:
# print Target_Output
# plt.subplot(2,2,1)
# plt.imshow(np.reshape(Input[0][0], (4,4)))
# plt.subplot(2,2,2)
# plt.imshow(np.reshape(Input[0][1], (4,4)))
# plt.subplot(2,2,3)
# plt.imshow(np.reshape(Input[0][2], (4,4)))
# plt.subplot(2,2,4)
# plt.imshow(np.reshape(Input[0][3], (4,4)))